In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
from data_handle import *

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def conv_model(n_input, n_output, n_enc_units, n_dec_units):
    root_word_input = Input(shape=(13, 309, 1), name="root_word_input")
    
    x = Conv2D(16, (3, 3), padding='same', activation='relu')(root_word_input)
    x = MaxPooling2D(2, 2)(x)
    x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(2, 2)(x)
    
    x = Flatten()(x)

    state_h = Dense(n_dec_units, activation='relu')(x)
    
    decoder_inputs = Input(shape=(None, 309), name="target_word_input")
    decoder_gru = GRU(n_dec_units, return_sequences=True, return_state=True, name="decoder_gru")
    decoder_outputs, _= decoder_gru(decoder_inputs, initial_state=state_h)
    
    decoder_dense = Dense(309, activation='softmax', name="train_output")
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([root_word_input, decoder_inputs], decoder_outputs)
    encoder_model = Model(root_word_input, state_h)
    
    decoder_state_input_h = Input(shape=(n_dec_units,))
    decoder_outputs, state_h= decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [decoder_outputs, state_h])

    return model, encoder_model, decoder_model


In [3]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()

n_chars = 11 + 2
n_features = len(char2int)
batch_size = 128
embed_size = 128

In [4]:
gen = generate_word_images(vocab, char2int, batch_size)

In [5]:
train, infenc, infdec = conv_model(13, 13, embed_size, embed_size)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [6]:
train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
root_word_input (InputLayer)    (None, 13, 309, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 13, 309, 16)  160         root_word_input[0][0]            
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 6, 154, 16)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 6, 154, 8)    1160        max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

In [7]:
n_batches = len(vocab) // batch_size
history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 3)

Epoch 1/3
2326/2326 [==============================] - 288s 124ms/step - loss: 0.3543 - acc: 0.9491
Epoch 2/3
2326/2326 [==============================] - 281s 121ms/step - loss: 0.0448 - acc: 0.9972
Epoch 3/3
   6/2326 [..............................] - ETA: 4:44 - loss: 0.0144 - acc: 0.9988

KeyboardInterrupt: 

In [10]:
embeddings = []
word2int = {}
i = 0
print(len(vocab))
for word in vocab:
    word2int[word] = len(word2int)
    vec = word2vec(char2int, word, 13).reshape((1, 13, 309, 1))
    emb = infenc.predict(vec)
    embeddings.append(emb)
    if i % 10000 == 0:
        print(i * 100.0 / len(vocab))
    i+=1

297836
0.0
3.3575524785452395
6.715104957090479
10.072657435635719
13.430209914180958
16.787762392726197
20.145314871271438
23.50286734981668
26.860419828361916
30.217972306907157
33.575524785452394
36.93307726399764
40.290629742542876
43.64818222108811
47.00573469963336
50.363287178178595
53.72083965672383
57.07839213526908
60.435944613814314
63.79349709235955
67.15104957090479
70.50860204945003
73.86615452799528
77.22370700654051
80.58125948508575
83.938811963631
87.29636444217623
90.65391692072147
94.01146939926672
97.36902187781195


In [11]:
from gensim_wrapper import *

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
embeds = np.stack(embeddings)

In [29]:
gensim = GensimWrapper(embed_size, 0, log=True)
embeds = embeds.reshape((-1, 128))
norms = np.linalg.norm(embeds, axis=1, keepdims=True)
embeds = embeds / norms

2018-10-23 01:26:49,604 : INFO : collecting all words and their counts
2018-10-23 01:26:49,605 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-10-23 01:26:49,690 : INFO : PROGRESS: at sentence #10000, processed 150787 words, keeping 38040 word types
2018-10-23 01:26:49,751 : INFO : PROGRESS: at sentence #20000, processed 309265 words, keeping 61588 word types
2018-10-23 01:26:49,841 : INFO : PROGRESS: at sentence #30000, processed 477795 words, keeping 81867 word types
2018-10-23 01:26:49,917 : INFO : PROGRESS: at sentence #40000, processed 647737 words, keeping 100652 word types
2018-10-23 01:26:49,985 : INFO : PROGRESS: at sentence #50000, processed 805941 words, keeping 116075 word types
2018-10-23 01:26:50,074 : INFO : PROGRESS: at sentence #60000, processed 967897 words, keeping 129675 word types
2018-10-23 01:26:50,133 : INFO : PROGRESS: at sentence #70000, processed 1144759 words, keeping 143321 word types
2018-10-23 01:26:50,202 : INFO : PROGRESS

In [30]:
gensim.set_embeddings(word2int, embeds)

2018-10-23 01:27:01,654 : INFO : precomputing L2-norms of word weight vectors


In [31]:
gensim.evaluate()

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
2018-10-23 01:27:04,411 : INFO : semantic word embedding: 0.0% (0/185)
2018-10-23 01:27:04,412 : INFO : total: 0.0% (0/185)
2018-10-23 01:27:04,929 : INFO : syntactic evaluation: 6.8% (9/132)
2018-10-23 01:27:04,929 : INFO : total: 6.8% (9/132)


76.14678899082568


In [32]:
from utils import *

utils = Utils(word2int, embeds)

In [33]:
utils.sorted_sim("ኢትዮጵያን")

[('ኢትዮጵያን', 1.0000001),
 ('አትዮጵያን', 0.99984026),
 ('ኢትዮጵያና', 0.9996797),
 ('ኢትዮጵያንን', 0.99963903),
 ('ኢትዮጵያያን', 0.9996242),
 ('ኢትዮጵያንና', 0.99951124),
 ('ኢትዮዮጵያና', 0.9989107),
 ('ኢትዮጵያኖች', 0.9988862),
 ('አትዮጵያ', 0.9987936),
 ('ኢትዮጵያ', 0.9987365)]